In [1]:
# !pip install gradio --upgrade

In [2]:
import numpy  as np
import pandas as pd
import gradio as gr

# visualization libraries
import matplotlib.pyplot as plt

import cv2
import torch
import torch.nn.functional as F
import uuid

# Live WebCam Demo Inception-V3

In [3]:
%pwd

'c:\\Users\\kings\\OneDrive - MMU\\MSC DATA SCIENCE\\facial_expression\\notebooks'

In [4]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 3)
state_dict = torch.load("../models/Inception3_Adamproposed_augmediapipe.pt")

model.load_state_dict(state_dict)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

Using cache found in C:\Users\kings/.cache\torch\hub\pytorch_vision_v0.10.0


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [ ]:
# def predict_emotion(input_video):
#     font_scale = 1.5
#     font = cv2.FONT_HERSHEY_PLAIN
#     cap = cv2.VideoCapture(input_video)
    
#     if not cap.isOpened():
#         raise IOError("Cannot open video")

#     # Video specifications
#     fps = int(cap.get(cv2.CAP_PROP_FPS))
#     desired_fps = fps // 2
#     width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) // 2
#     height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 2
#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')

#     # Generate unique filename using uuid
#     output_video_name = f"output_{uuid.uuid4()}.mp4"
#     out = cv2.VideoWriter(output_video_name, fourcc, desired_fps, (width, height))
    
#     facedet = cv2.CascadeClassifier('../haarcascades/haarcascade_frontalface_default.xml')

#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         faces = facedet.detectMultiScale(gray_frame, 1.1, 4)

#         for x, y, w, h in faces:
#             roi_gray = gray_frame[y:y+h, x:x+w]
#             roi_color = frame[y:y+h, x:x+w]

#             # Prepare for model input
#             rgb_image = cv2.cvtColor(roi_gray, cv2.COLOR_GRAY2RGB)
#             image = cv2.resize(rgb_image, (299, 299))
#             image = image / 255.0
#             data = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float().to(device)

#             with torch.no_grad():
#                 outputs = model(data)
#                 pred = F.softmax(outputs[0], dim=-1)
#                 final_pred = torch.argmax(pred, 0)

#             # Map emotion prediction
#             emotion = ["Negative", "Neutral", "Positive"][final_pred]

#             # Draw bounding box and label
#             text_x, text_y = x, y - 10
#             cv2.putText(frame, emotion, (text_x, text_y), font, font_scale, (36, 255, 12), 2)
#             cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        
#         out.write(frame)  # Write the frame to the video file

#     # Release resources after processing
#     # cap.release()
#     out.release()
#     # return output_video_name  # Return the video file path

In [19]:
def predict_emotion(input_video):
    cap = cv2.VideoCapture(input_video)  
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 

    output_video_name = f"output_{uuid.uuid4()}.mp4"
    out = cv2.VideoWriter(output_video_name, fourcc, frame_rate, (frame_width, frame_height))

    while True:
        # Capture a frame
        ret, frame = cap.read()
        if not ret:
            break  # Exit the loop if no frame is captured

        facedet = cv2.CascadeClassifier('..\haarcascades\haarcascade_frontalface_default.xml')
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facedet.detectMultiScale(gray_frame, 1.1, 4)

        for x, y, w, h in faces:
            roi_gray = gray_frame[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            facess = facedet.detectMultiScale(roi_gray)
            if len(facess) == 0:
                continue
            else:
                for (ex, ey, ew, eh) in facess:
                    face_roi = roi_color[ey:ey+eh, ex:ex+ew]  # cropping the face

        rgb_image = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
        image = cv2.resize(rgb_image, (299, 299))
        image = image / 255.0  # normalization
        data = torch.from_numpy(image)
        data = data.type(torch.FloatTensor)
        data = data.to(device)
        reshaped_data = data.permute(2, 0, 1)  # Reshape the tensor to have 3 as the first dimension
        reshaped_data = reshaped_data.unsqueeze(0)  # add a fourth dimension
        outputs = model(reshaped_data)
        pred = F.softmax(outputs[0], dim=-1)
        final_pred = torch.argmax(pred, 0)

        if final_pred == 0:
            emotion = "Negative"
        elif final_pred == 1:
            emotion = "Neutral"
        else:
            emotion = "Positive"

        text_x, text_y = x, y - 10  # Adjust the position above the bounding box
        cv2.putText(frame, emotion, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

        # Write the frame to the video file
        out.write(frame)
    cap.release()
    out.release()

    # return output_video_name

In [20]:
# import os
# def video_interface(input_video):
#     # processed_video_path = predict_emotion(input_video)
#     # processed_video_path = os.path.join(os.getcwd(), processed_video_path)
#     processed_video_path = predict_emotion(input_video)
#     with open(processed_video_path, 'rb') as f:
#         return f.read()

In [21]:
def video_interface(input_video):
    # Return a known, pre-existing video for testing
    processed_video_path = predict_emotion(input_video)
    test_video_path = 'output_873d5bee-b407-4d1e-901c-a36e86dde300.mp4'
    return test_video_path


In [22]:
# !pip install gradio-webrtc

In [23]:
# import gradio as gr
# from gradio_webrtc import WebRTC

# css = """.my-group {max-width: 600px !important; max-height: 600px !important;}
#          .my-column {display: flex !important; justify-content: center !important; align-items: center !important;}"""

# with gr.Blocks(css=css) as demo:
#     gr.HTML(
#         """
#         <h1 style='text-align: center'>
#         YOLOv10 Webcam Stream (Powered by WebRTC ⚡️)
#         </h1>
#         """
#     )
#     with gr.Column(elem_classes=["my-column"]):
#         with gr.Group(elem_classes=["my-group"]):
#             image = WebRTC(label="Stream", mode="send-receive", modality="video")
#         image.stream(
#             fn=predict_emotion, inputs=[image], outputs=[image], time_limit=10
#         )

# if __name__ == "__main__":
#     demo.launch()

In [25]:
with gr.Blocks(gr.themes.Glass()) as demo:
    gr.Markdown(
        """
    # FACIAL EXPRESSION PREDICTION

    ### QUICK GUIDE
    This application processes video sequences to analyze facial expressions and classify them into three main categories: Positive, Neutral, and Negative. Here are some key functionalities and tips to help you get the most out of this tool:
    1. Upload or capture a video using the **Video Source** section.
    2. The processed video output will be displayed in the **Processed Video** section, showing detected faces and their predicted expressions.

    ### HOW IT WORKS
    - The system uses face detection and deep learning models to recognize and classify facial expressions frame-by-frame.
    - Detected expressions are overlaid on the video, and real-time feedback is provided during processing.

    **NOTE**: The application is built to automatically process the video upon upload and display the results, including highlighted faces and corresponding emotion labels.
    """
    )
    with gr.Row():
        with gr.Column():
            video = gr.Video(label="Video Source")
        with gr.Column():
            output_video = gr.Video(label="Processed Video", autoplay=True)

    video.upload(fn=video_interface, inputs=[video], outputs=[output_video])

if __name__ == "__main__":
    demo.launch()


* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [30]:
# import gradio as gr

# # Gradio Interface Code
# def video_interface(input_video):
#     output_video_path = "processed_output.mp4"  # Save in the current working directory
#     output_video_path = predict_emotion(input_video, output_video_path)
#     return output_video_path

# # Use the same component for input and output
# video_component = gr.Video(label="Upload and Process Video")

# gr.Interface(
#     fn=video_interface,
#     inputs=video_component,
#     outputs=video_component,
#     title="Facial Expression Tracking",
#     description="Upload a video to see different emotions in action. The uploaded video will be replaced by the processed video upon submission."
# ).launch()


* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


End of video stream or read error.
Total frames processed: 81


In [ ]:
# 